In [1]:
import anthropic
import pandas as pd
from dotenv import load_dotenv
from pathlib import Path
import base64
import httpx
import ollama

In [2]:
# Load the environment file with the api key in it
load_dotenv()

True

In [2]:
import ollama

response = ollama.chat(
    model="llama3.2-vision",
    messages=[
        {
            "role": "user",
            "content": "Tell me an interesting fact about elephants",
        },
    ],
)
print(response["message"]["content"])

Here's one:

**Elephants have a highly developed sense of empathy and self-awareness.**

In fact, studies have shown that elephants are able to recognize themselves in mirrors, which is a cognitive ability shared by only a few other animals, including humans, dolphins, and chimpanzees. This level of self-awareness suggests that elephants have a sophisticated understanding of their own emotions and mental states.

But what's even more fascinating is that elephants also seem to be able to understand and respond to the emotional states of others. For example, they've been known to console each other after a death or injury, and will even go out of their way to help distressed family members.

One famous study involved a matriarchal elephant named Thembi, who was separated from her herd at a young age. When she was finally reunited with her family years later, the reunion was incredibly emotional - with all the elephants greeting each other with trumpeting and embracing. It's moments like 

In [2]:
df = pd.read_csv('ItemData.csv', index_col=0)
df.head()

,ProductName,Brand,Price,Category,Sizes,TextDescription,BulletPointDescription,ImageUrls
0,1017 ALYX 9SM Circle Logo Popover Hoodie,1017-alyx-9sm,325.0,Hoodies & Sweats,[],Drawing influences from California skate cultu...,"['100% Cotton', 'Drawstring Hood', 'Kangaroo P...","['https://media.endclothing.com/media/f_auto,q..."
0,1017 ALYX 9SM Side Zip Camo Pant,1017-alyx-9sm,443.0,Trousers,[],1017 ALYX 9SM innovative designs take cues fro...,"['100% Nylon', 'Elasticated Waist', 'Zip Closu...","['https://media.endclothing.com/media/f_auto,q..."
0,1017 ALYX 9SM x Blackmeans Denim Jacket,1017-alyx-9sm,779.0,Coats & Jackets,[],Working together on a capsule that creates a r...,"['100% Cotton', 'Classic Collar', 'Button Clos...","['https://media.endclothing.com/media/f_auto,q..."
0,1017 ALYX 9SM Oversized Camo Nylon Bomber Jacket,1017-alyx-9sm,779.0,Coats & Jackets,[],With an unmistakable experimental streetwear a...,"['100% Nylon', 'Stand Up Collar', 'Snap Button...","['https://media.endclothing.com/media/f_auto,q..."
0,1017 ALYX 9SM Nylon Logo Varsity Jacket,1017-alyx-9sm,489.0,Coats & Jackets,[],"Matthew Williams’s genre-blurring, future-orie...","['100% Polyester', 'Shawl Collar', 'Snap Butto...","['https://media.endclothing.com/media/f_auto,q..."


In [3]:
client = anthropic.Anthropic()

image_media_type = "image/jpeg"
model="claude-3-5-sonnet-20241022"
max_tokens=1024

# Create the json message to be inputted into Claude API for an image url
def create_json_message_for_image(image_url):

    image_data = base64.standard_b64encode(httpx.get(image_url).content).decode("utf-8")

    image_dict = {
                "type": "image",
                "source": {
                    "type": "base64",
                    "media_type": image_media_type,
                    "data": image_data,
                },
            }
    
    return image_dict

# Gets description of item by using Claude 3.5 Sonnet on image urls
def get_image_description_for_row(df_row: pd.Series):

    # print(df_row)

    try:

        category = df_row['Category']
        image_urls = df_row['ImageUrls'].replace('[', '').replace(']', '').replace("'", '').split(', ')

        content = [create_json_message_for_image(url) for url in image_urls if 'http' in url]
        input_message = "These are images of an item from an ecommerce website in the category " + category + ".\n Describe the item in detail. Begin by identifying the type and category, then specify the material, main color, and any secondary colors or patterns. Describe the fit and style and include any notable design features. Add the typical occasion or style it's intended for, and any other unique characteristics."

        text_message = {
                            "type": "text",
                            "text": input_message
                        }

        content.append(text_message)

    # print(content)
    
        message = client.messages.create(
            model=model,
            max_tokens=max_tokens,
            messages=[
                {
                    "role": "user",
                    "content": content,
                }
            ],
        )

        return message.content[0].text
    
    except:
        print("Error with item: ", df_row["ProductName"])
        return "error"
    
# Gets description of item by using Claude 3.5 Sonnet on image urls
def get_text_description_for_row(df_row: pd.Series):

    # print(df_row)

    try:

        category = df_row['Category']
        product_name = df_row['ProductName']
        brand = df_row['Brand']
        price = df_row['Price']
        text_description = df_row['TextDescription']
        bullet_points = df_row

        input_message = "These are images of an item from an marketplace website in the category " + category + ".\n Describe the item in detail. Begin by identifying the type and category, then specify the material, main color, and any secondary colors or patterns. Describe the fit and style and include any notable design features. Add the typical occasion or style it's intended for, and any other unique characteristics."

        text_message = {
                            "type": "text",
                            "text": input_message
                        }

        content.append(text_message)

    # print(content)
    
        message = client.messages.create(
            model=model,
            max_tokens=max_tokens,
            messages=[
                {
                    "role": "user",
                    "content": content,
                }
            ],
        )

        return message.content[0].text
    
    except:
        print("Error with item: ", df_row["ProductName"])
        return "error"

In [6]:
len(image_descriptions)

78

In [4]:
image_descriptions = []

for i in range(len(df)):

    if i % 500 == 0:
        print(i)

    try:
        image_descriptions.append(get_image_description_for_row(df.iloc[i]))
    except:
        image_descriptions.append("Error with: " + str(i))

0
Error with item:  1017 ALYX 9SM 9SM T-Shirt
Error with item:  66° North Blaer 66°N Chest Logo T-Shirt
Error with item:  A Bathing Ape ABC Camo Rubber Coaster
Error with item:  A Bathing Ape Bape Stripe Socks
Error with item:  A Bathing Ape Cursive College Logo T-Shirt
Error with item:  A-COLD-WALL* Ando Cargo Pant
Error with item:  A.P.C. Kerlouan Wool Work Jacket
Error with item:  AAPE French Terry Sweat Short
Error with item:  Acne Studios Edden Logo Wings Long Sleeve T-Shirt
Error with item:  Acne Studios Sandrok Stripe AS Short Sleeve Shirt
Error with item:  Adidas x Fear Of God Athletics Alaska Hoodie
Error with item:  Adidas x Fear Of God Athletics Woven Relaxed Track Pant
500
Error with item:  Adidas x Fear Of God Athletics Logo 3/4 T-Shirt
Error with item:  Adidas x Fear Of God Athletics Heavy Fleece Pullover Crew Ne
Error with item:  Adidas Adizero Aruku Sneakers
Error with item:  Adidas CLOT by EC Ribbed Tank
Error with item:  Adidas Handball Spezial
Error with item:  Adida

In [5]:
len(image_descriptions)

14912

In [6]:
df = pd.DataFrame(image_descriptions)

In [8]:
df.head()

,0
0,This is a black pullover hoodie with a distinc...
1,This appears to be Military-Style Cargo Pants ...
2,This is a textured denim jacket with a unique ...
3,This is a Military-Style Bomber Jacket in a Mu...
4,This is a men's bomber/varsity style jacket in...


In [9]:
df.to_csv('ImageDescriptions.csv')

In [17]:
for i in range(len(df)-1, 0, -1):
    if df.iloc[i].values[0] != 'error':
        print(i)
        break

6232


In [16]:
df.iloc[0].values[0]

'This is a black pullover hoodie with a distinctive graphic design element. The key features include:\n\nType & Category:\n- Pullover hoodie\n- Casual streetwear\n\nMaterial & Color:\n- Appears to be made of cotton or cotton-blend sweatshirt material\n- Primary color is black\n- Contrasting white/off-white graphic elements\n\nDesign Features:\n- Large circular graphic pattern on the front featuring "X" marks and other geometric shapes arranged in a face-like circular design\n- Full hood with drawstrings\n- Traditional kangaroo pocket in front\n- Ribbed cuffs and hem\n- Front zipper closure\n- Angular letter details visible on the front\n\nStyle & Fit:\n- Appears to have a relaxed, comfortable fit\n- Contemporary streetwear aesthetic\n- Mid-hip length\n- Standard hoodie silhouette\n\nIntended Use:\n- Casual everyday wear\n- Streetwear/urban fashion\n- Could be worn for leisure or casual outings\n\nThe design has a modern, graphic-focused aesthetic that would appeal to those interested i